In [1]:
import pandas as pd
import csv

In [2]:
tabla_series = pd.read_csv("./components/info_gustos_series.csv", encoding="utf-8")
tabla_movies = pd.read_csv("./components/info_gustos_peliculas.csv", encoding="utf-8")
tabla_musica = pd.read_csv("./components/info_gustos_canciones.csv", encoding="utf-8")
tabla_libros = pd.read_csv("./components/info_gustos_libros.csv", encoding="utf-8")
tabla_series.columns = ["Nombre", "Carrera", "Edad", "Series", "Genre"]
tabla_movies.columns = ["Nombre", "Carrera", "Edad", "Pelis", "Genre"]
tabla_musica.columns = ["Nombre", "Carrera", "Edad", "Musica", "Artista", "Genre"]
tabla_libros.columns = ["Nombre", "Carrera", "Edad", "Libro"]

# print(tabla_musica)

peliculas = tabla_movies["Pelis"]
series = tabla_series["Series"]
musica = tabla_musica["Musica"]
libros = tabla_libros["Libro"]
print(libros)
# print(musica)

0                        Libros
1          El-nombre-del-viento
2            La-magia-del-grial
3      The-Lies-of-locke-Lamora
4                 Dresden-files
                ...            
78            El-arte-de-guerra
79               Un-mundo-Feliz
80            El-Psicoanalista 
81    El-Retrato-de-Dorian-Grey
82         El-arte-de-la-guerra
Name: Libro, Length: 83, dtype: object


In [3]:
# Displaying final output format
from IPython.display import display, Markdown, Latex
# LangChain Dependencies
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
# from langchain_community.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langgraph.graph import START, END, StateGraph
# For State Graph 
from typing_extensions import TypedDict
import json
import os

In [4]:
# Defining LLM
local_llm = 'llama3'
llama3 = OllamaLLM(model=local_llm, temperature=0)
llama3_json = OllamaLLM(model=local_llm, format='json', temperature=0)

In [5]:
# Generation Prompt

media_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all tv shows and movies, you will be given an array of may tv shows, and your job is to return
    a your response with the format JSON: {{"NameOfTheShow": "genre"}} with only one genre for each show.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {shows} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["shows"],
)

series_chain = media_analizer | llama3_json | JsonOutputParser()


In [6]:
# Generation Prompt

music_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of music, you will be given a song, and your job is to return
    a your response with the format JSON: {{"NameOfTheSong": "genre"}} with only one genre for each song.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Shows: {song} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["song"],
)

music_chain = music_analizer | llama3_json | JsonOutputParser()

In [7]:
# Generation Prompt

book_analizer = PromptTemplate(
    template="""
    
    <|begin_of_text|>
    
    <|start_header_id|>system<|end_header_id|> 
    
    You are an expert in all kind of books, you will be given a book, and your job is to return
    a your response with the format JSON: {{"NameOfTheBook": "genre"}} with only one genre for each book.
    Do not give any further explanation on your response.
    Try using general purpose genres, for example, sitcom and comedy are the same.
    
    <|eot_id|>
    
    <|start_header_id|>user<|end_header_id|>
    
    Book: {book} 
    
    <|eot_id|>
    
    <|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["book"],
)

book_chain = book_analizer | llama3_json | JsonOutputParser()

In [8]:
# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    shows = state["shows"]
    generation = series_chain.invoke({"shows": shows})

    # Answer Generation

    return {"generation": generation}

In [9]:
# Node - Generate

def generate_music(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    song = state["song"]
    # print(song)

    # Answer Generation
    generation = music_chain.invoke({"song": song})

    return {"generation": generation}

In [10]:
def generate_books(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    # print("Step: Generating Final Response")
    book = state["book"]
    # print(book)

    # Answer Generation
    generation = book_chain.invoke({"book": book})

    return {"generation": generation}

In [11]:
# Graph State
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    shows: str
    song: str
    book: str
    media: str
    generation : str

In [21]:
def router(state):
  print(state["media"])
  return "book"

In [25]:
# Build the nodes
workflow = StateGraph(GraphState)
workflow.add_node("generate_shows", generate)
workflow.add_node("generate_music", generate_music)
workflow.add_node("generate_books", generate_books)

workflow.set_conditional_entry_point(
  router,
  {
    "shows": "generate_shows",
    "music": "generate_music",
    "book": "generate_books",
  }
)

workflow.add_edge("generate_shows", END)
workflow.add_edge("generate_music", END)
workflow.add_edge("generate_books", END)

# Compile the workflow
local_agent = workflow.compile()


In [26]:
def run_agent_series():
    shows = []
    for serie in series:
        output = local_agent.invoke({"media": "shows", "media": "shows", "shows": serie})
        # print("=======")
        print(output["generation"])
        shows.append(output["generation"])
    return shows

def run_agent_movies():
    movies = []
    for movie in peliculas:
        output = local_agent.invoke({"media": "shows", "shows": movie})
        # print("=======")
        print(output["generation"])
        movies.append(output["generation"])

    return movies    

def run_agent_music():
    music = []
    for song in musica:
        output = local_agent.invoke({"media": "music", "song": song})
        print(output["generation"])
        music.append(output["generation"])
    
    return music

def run_agent_book():
    books = []
    for book in libros:
        output = local_agent.invoke({"media": "book", "book": book})
        print(output["generation"])
        books.append(output["generation"])
    
    return books

In [37]:
# all_shows = run_agent_series()

In [38]:
# all_movies = run_agent_movies()

In [39]:
# all_music = run_agent_music()

In [27]:
all_books = run_agent_book()

book
{'Libros': 'Non-Fiction'}
book
{'El-nombre-del-viento': 'Fiction'}
book
{'La Magia del Grial': 'Fantasy'}
book
{'The Lies of Locke Lamora': 'Fantasy'}
book
{'The Dresden Files': 'Urban Fantasy'}
book
{'The False Prince': 'Young Adult Fiction'}
book
{'El Principito': 'Children'}
book
{"Ms. Peregrine's Home for Peculiar Children": 'Fantasy'}
book
{'The Hunger Games': 'Science Fiction'}
book
{'Las Crónicas Lunares': 'Science Fiction'}
book
{'Persona': 'Psychology'}
book
{'The Seven Husbands of Evelyn Hugo': 'Romance'}
book
{'1984': 'Dystopian'}
book
{'Prometeo-Americano': 'Mystery'}
book
{'Rebelión-En-La-Granja': 'Science Fiction'}
book
{'Paul McCartney: The Life': 'Biography'}
book
{'Baumgartner': 'Memoir'}
book
{'Frankenstein': 'Horror'}
book
{'Anna Karenina': 'Romance'}
book
{'El-Castillo': 'Fantasy'}
book
{'Hábitos-Atomicos': 'Self-Help'}
book
{'Encuentra-tu-persona-vitamina': 'Self-Help'}
book
{'El-psicoanalista': 'Psychology'}
book
{'A Soul to Keep': 'Historical Fiction'}
book


In [28]:
df = pd.DataFrame([{"Book": k, "Genre": v} for s in all_books for k, v in s.items()])
print(df)
df.to_csv("dict_books.csv", index=False)


                          Book                  Genre
0                       Libros            Non-Fiction
1         El-nombre-del-viento                Fiction
2           La Magia del Grial                Fantasy
3     The Lies of Locke Lamora                Fantasy
4            The Dresden Files          Urban Fantasy
..                         ...                    ...
78        El arte de la guerra            Non-fiction
79              Un-mundo-Feliz                Mystery
80            El Psicóanalista             Psychology
81  The Picture of Dorian Gray  Philosophical Fiction
82        El arte de la guerra            Non-fiction

[83 rows x 2 columns]


In [ ]:
# df = pd.DataFrame([{"Show": k, "Genre": v} for s in all_shows for k, v in s.items()])
# print(df)
# df.to_csv("dict_series.csv", index=False)
# df = pd.DataFrame([{"Movie": k, "Genre": v} for s in all_movies for k, v in s.items()])
# print(df)
# df.to_csv("dict_movies.csv", index=False)

In [ ]:
#References:
# Pandas: https://pandas.pydata.org/docs/getting_started/index.html
# Langchain: https://langchain-ai.github.io/langgraph/tutorials/workflows/#prompt-chaining 